In [29]:
# BOILER PLATE, MUST BE RUN ON SUBMIT NODE
%load_ext autoreload
%autoreload 2
import socket
print("Running on host: ", socket.gethostname())

import sys 
lib_path = '/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws'
if lib_path not in sys.path:
    sys.path.append(lib_path)
#=================================================
from typing import Any 
import time

import pandas as pd 
import json
import os
import wandb 
import launch_evals

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Running on host:  submit-1


# Eval runner

In [39]:
def run_eval_sweep(
    configs_path: str,
    watch_path: str,
    max_jobs: int,
    wait_seconds: int,
    conda_env: str | None = None,
    prefix_name: str | None = None,
    start_step: int | None = None,
    stop_step: int | None = None,
    every_steps: int | None = None,
    base_eval_output_dir: str = "evals",
    overrides: list[list[tuple[str, Any]]] | None = None,
    rerun_if_exists: bool = False,
    wandb_project: str | None = None
):
    num_jobs = 0
    
    available_steps = launch_evals._get_available_checkpoints(watch_path)
    
    for step in available_steps:
        if (start_step is not None and step < start_step) or (
            every_steps is not None and step % every_steps != 0
        ):
            continue
        
        new_jobs = launch_evals.run_eval_steps(
            configs_path=configs_path,
            watch_path=watch_path,
            max_num_jobs=max_jobs - num_jobs,
            conda_env=conda_env,
            prefix_name=prefix_name,
            steps=[step],
            base_eval_output_dir=base_eval_output_dir,
            overrides=overrides,
            rerun_if_exists=rerun_if_exists,
        )
        
        num_jobs += new_jobs
        if num_jobs == max_jobs:
            # will not launch more job in this iteration
            break

        if stop_step is not None and step == stop_step:
            break
        
    return num_jobs

with open('eval_watcher_config.json', 'r') as file:
    config = json.load(file)

global_config = config.copy()
global_config.pop("watch_jobs")

for job in config['watch_jobs']:
    job_config = global_config.copy()
    job_config.update(job)
    
    print(job_config)
    print(f"Scanning for new checkpoints in {job_config['watch_path']}")
    num_jobs = run_eval_sweep(**job_config)
    print(num_jobs)
    
time.sleep(global_config["wait_seconds"])

{'wait_seconds': 900, 'wandb_project': 'mm10.1_stage1', 'configs_path': '/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm10.1/stage1/eval_overwrite', 'max_jobs': 2, 'watch_path': '/fsx_0/checkpoints/mm10.1/MM10.1_Stage1_70B/MH22final_70B_BigG_336px_R1'}
Scanning for new checkpoints in /fsx_0/checkpoints/mm10.1/MM10.1_Stage1_70B/MH22final_70B_BigG_336px_R1


0
{'wait_seconds': 900, 'wandb_project': 'mm10.1_stage1', 'configs_path': '/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm10.1/stage1/eval_overwrite', 'max_jobs': 2, 'watch_path': '/fsx_0/checkpoints/mm10.1/MM10.1_Stage1_70B/MH22final_70B_ViTH_336px_R1'}
Scanning for new checkpoints in /fsx_0/checkpoints/mm10.1/MM10.1_Stage1_70B/MH22final_70B_ViTH_336px_R1
2


KeyboardInterrupt: 

# Fix VQA

In [4]:
import torch
from torch.utils.data import Dataset, DataLoader, Subset

# Step 1: Define a custom dataset
class DummyDataset(Dataset):
    def __init__(self):
        # Let's assume ids are just numbers from 0 to 99
        self.ids = list(range(100))

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, index):
        # Return the id and a dummy tensor data (for example purposes)
        return self.ids[index], torch.tensor([1.0, 2.0, 3.0])

# Step 2: Create the dataset and dataloader
dataset = DummyDataset()
eval_dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

# Step 3: Access the length of the dataset before sampling
length_eval_dataloader_before_sampling = len(eval_dataloader.dataset.ids)
print("Length of eval dataloader before sampling:", length_eval_dataloader_before_sampling)

Length of eval dataloader before sampling: 100


In [9]:
len(dataset.ids)

100

In [7]:
subset_dataset = Subset(dataset, indices=range(50))
len(subset_dataset.ids)

AttributeError: 'Subset' object has no attribute 'ids'

# Wandb publish

In [19]:
def create_wandb_eval_report(
    report_df: pd.DataFrame,
    wandb_project: str,
    checkpoint_watch_path: str,
    wandb_entity: str="sg_share_ai",
    eval_base_dir: str="evals",
    delete_if_exists: bool = True
):
    wandb.login()
    
    # look for existing run
    prev_run_config = None
    wandb_run_config_file = os.path.join(checkpoint_watch_path, eval_base_dir, "wandb_report_run.json")
    if delete_if_exists and os.path.isfile(wandb_run_config_file):
        print(f"Deleting existing run at {wandb_run_config_file}")
        with open(wandb_run_config_file, 'r') as f:
            prev_run_config = json.load(f)
            try:
                api = wandb.Api()
                prev_run = api.run(f"{prev_run_config['entity']}/{prev_run_config['project']}/{prev_run_config['id']}")
                prev_run.delete()
            except Exception as e:
                print(f"Unable to delete previous run. {e}")
                
    checkpoint_folder = os.path.basename(checkpoint_watch_path)
    
    run = wandb.init(
        entity=wandb_entity,
        project=wandb_project,
        name=f"{checkpoint_folder}_{eval_base_dir}"
    )
    
    for step, row in report_df.iterrows():
        wandb.log(dict(row), step=step)
    
    # Saving run config to eval folder
    run_config = {
        "entity": run.entity,
        "id": run.id,
        "project": run.project,
        "name": run.name
    }
    
    with open(wandb_run_config_file, 'w') as f:
        json.dump(run_config, f)
    
    wandb.finish()
    
create_wandb_eval_report(
    checkpoint_watch_path="/fsx_0/checkpoints/mm10.1/MM10.1_Stage1_70B/MH19_70B_BigG_336px_R1_test",
    wandb_project="mm10.1_stage1",
    report_df=df
)


Deleting existing run at /fsx_0/checkpoints/mm10.1/MM10.1_Stage1_70B/MH19_70B_BigG_336px_R1_test/evals/wandb_report_run.json


mmmu,▂▂▁▄▅▅▅█▅▄▅▄▆▆▅▅▅▆▇
ai2d,nan
chartqa,nan
docvqa,nan
infographics,nan
infographics_w_ocr,nan
mathvista,nan
mmbench,nan
mmmu,56.78
vqa,nan


# MM10.1 - Stage 1

In [23]:
watch_path="/fsx_0/checkpoints/mm10.1/MM10.1_Stage1_70B/MH19_70B_BigG_336px_R1_test"

df = launch_evals.read_eval_results(
    watch_path=watch_path,
    read_format="report_mmai"
)

create_wandb_eval_report(
    checkpoint_watch_path=watch_path,
    wandb_project="mm10.1_stage1",
    report_df=df
)


Deleting existing run at /fsx_0/checkpoints/mm10.1/MM10.1_Stage1_70B/MH19_70B_BigG_336px_R1_test/evals/wandb_report_run.json


mmmu,▁
ai2d,nan
chartqa,nan
docvqa,nan
infographics,nan
infographics_w_ocr,nan
mathvista,nan
mmbench,nan
mmmu,44.22
vqa,nan


In [25]:
watch_path="/fsx_0/checkpoints/mm10.1/MM10.1_Stage1_70B/MH22_70B_BigG_336px_R1_test"

df = launch_evals.read_eval_results(
    watch_path=watch_path,
    read_format="report_mmai"
)

create_wandb_eval_report(
    checkpoint_watch_path=watch_path,
    wandb_project="mm10.1_stage1",
    report_df=df
)


mmmu,▂▂▁▄▅▅▅█▅▄▅▄▆▆▅▅▅▆▇
ai2d,nan
chartqa,nan
docvqa,nan
infographics,nan
infographics_w_ocr,nan
mathvista,nan
mmbench,nan
mmmu,56.78
vqa,nan


# Test py launcher

In [229]:
launch_evals.run_eval_steps(
    configs_path="/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm10/stage2/eval_overwrite_fix",
    watch_path="/fsx_0/checkpoints/mm10/MM10-Stage2-70B/MH21_70B_336px_exp30d_docvqa_unfreeze_lr1_n128",
    steps=[15000],
    base_eval_output_dir="evals_py_fix",
    max_num_jobs=10
)

10

In [237]:
launch_evals.read_eval_results(
    watch_path="/fsx_0/checkpoints/mm10/MM10-Stage2-70B/MH21_70B_336px_exp30d_docvqa_unfreeze_lr1_n128",
    base_eval_output_dir="evals_py_fix",
    read_format="report_mmai",
    steps=[15000]
)

,mmmu,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench
15000,NaN,73.77,48.8,78.01,50.69,NaN,72.08,64.92,59.13,73.19


In [230]:
launch_evals.read_eval_results(
    watch_path="/fsx_0/checkpoints/mm10/MM10-Stage2-70B/MH21_70B_336px_exp30d_docvqa_unfreeze_lr1_n128",
    base_eval_output_dir="evals_py",
    read_format="report_mmai",
    steps=[15000]
)

,mmmu,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench
15000,56.44,72.8,48.8,78.01,52.22,NaN,72.08,64.92,59.13,72.24


In [22]:
glob("/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm10.1/stage1/eval_overwrite/eval_*.json", recursive=True)

['/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm10.1/stage1/eval_overwrite/eval_mmmu.json',
 '/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm10.1/stage1/eval_overwrite/eval_chartqa.json',
 '/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm10.1/stage1/eval_overwrite/eval_infographics_w_ocr.json',
 '/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm10.1/stage1/eval_overwrite/eval_docvqa.json',
 '/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm10.1/stage1/eval_overwrite/eval_ai2d.json',
 '/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm10.1/stage1/eval_overwrite/eval_textvqa.json',
 '/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm10.1/stage1/eval_overwrite/eval_mmbench.json',
 '/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm10.1/stage1/eval_overwrite/eval_infographics.json',
 '/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm10.1/stage1/eval_overwrite/eval_mathvista.json',
 '/fsx_0/user/tranx/rsync/llm_mm_al

In [5]:
from glob import glob 
import os

In [13]:
path = "/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm10.1/stage1/eval_overwrite/*"
os.path.isdir(path)

False

In [18]:
glob("/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm10.1/stage1/eval_overwrite/*", recursive=True)

['/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm10.1/stage1/eval_overwrite/eval_mmmu.json',
 '/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm10.1/stage1/eval_overwrite/eval_chartqa.json',
 '/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm10.1/stage1/eval_overwrite/eval_infographics_w_ocr.json',
 '/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm10.1/stage1/eval_overwrite/eval_docvqa.json',
 '/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm10.1/stage1/eval_overwrite/eval_ai2d.json',
 '/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm10.1/stage1/eval_overwrite/eval_textvqa.json',
 '/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm10.1/stage1/eval_overwrite/eval_mmbench.json',
 '/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm10.1/stage1/eval_overwrite/eval_infographics.json',
 '/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm10.1/stage1/eval_overwrite/eval_mathvista.json',
 '/fsx_0/user/tranx/rsync/llm_mm_al

In [3]:
glob("/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm10.1/stage1/eval_overwrite/eval_mmmu.json", recursive=True)

['/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm10.1/stage1/eval_overwrite/eval_mmmu.json']

In [17]:
from glob import glob 
glob("/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm10.1/stage1/eval_overwrite/*", recursive=True)

['/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm10.1/stage1/eval_overwrite/eval_mmmu.json',
 '/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm10.1/stage1/eval_overwrite/eval_chartqa.json',
 '/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm10.1/stage1/eval_overwrite/eval_infographics_w_ocr.json',
 '/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm10.1/stage1/eval_overwrite/eval_docvqa.json',
 '/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm10.1/stage1/eval_overwrite/eval_ai2d.json',
 '/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm10.1/stage1/eval_overwrite/eval_textvqa.json',
 '/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm10.1/stage1/eval_overwrite/eval_mmbench.json',
 '/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm10.1/stage1/eval_overwrite/eval_infographics.json',
 '/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm10.1/stage1/eval_overwrite/eval_mathvista.json',
 '/fsx_0/user/tranx/rsync/llm_mm_al

# Inspect config

In [221]:
!cat /fsx_0/checkpoints/mm10/MM10-Stage2-70B/MH21_70B_336px_exp30d_docvqa_unfreeze_lr1_n128/evals_py/eval_vqa_checkpoint-18800/*/run_log.txt | grep "lm_mh_tokenizer_version"

eval args: {'output_dir': '/fsx_0/checkpoints/mm10/MM10-Stage2-70B/MH21_70B_336px_exp30d_docvqa_unfreeze_lr1_n128/evals_py/eval_vqa_checkpoint-18800', 'overwrite_output_dir': True, 'do_train': True, 'do_eval': False, 'do_predict': False, 'eval_strategy': 'no', 'prediction_loss_only': False, 'per_device_train_batch_size': 64, 'per_device_eval_batch_size': 48, 'per_gpu_train_batch_size': None, 'per_gpu_eval_batch_size': None, 'gradient_accumulation_steps': 2, 'eval_accumulation_steps': None, 'eval_delay': 0, 'learning_rate': 5e-05, 'weight_decay': 0.1, 'adam_beta1': 0.9, 'adam_beta2': 0.95, 'adam_epsilon': 1e-08, 'max_grad_norm': 1.0, 'num_train_epochs': 1.0, 'max_steps': -1, 'lr_scheduler_type': 'cosine', 'lr_scheduler_kwargs': {}, 'warmup_ratio': 0.0, 'warmup_steps': 200, 'log_level': 'info', 'log_level_replica': 'info', 'log_on_each_node': True, 'logging_dir': '/fsx_0/checkpoints/mm10/MM10-Stage2-70B/MH21_70B_336px_exp30d_docvqa_unfreeze_lr1_n128/evals_py/eval_vqa_checkpoint-18800', '

In [222]:
!cat /fsx_0/checkpoints/mm10/MM10-Stage2-70B/MH21_70B_336px_exp30d_docvqa_unfreeze_lr1_n128/evals_py/eval_ai2d_checkpoint-18800/*/run_log.txt | grep "instruction_model_type"

eval args: {'output_dir': '/fsx_0/checkpoints/mm10/MM10-Stage2-70B/MH21_70B_336px_exp30d_docvqa_unfreeze_lr1_n128/evals_py/eval_ai2d_checkpoint-18800', 'overwrite_output_dir': True, 'do_train': True, 'do_eval': False, 'do_predict': False, 'eval_strategy': 'no', 'prediction_loss_only': False, 'per_device_train_batch_size': 64, 'per_device_eval_batch_size': 48, 'per_gpu_train_batch_size': None, 'per_gpu_eval_batch_size': None, 'gradient_accumulation_steps': 2, 'eval_accumulation_steps': None, 'eval_delay': 0, 'learning_rate': 5e-05, 'weight_decay': 0.1, 'adam_beta1': 0.9, 'adam_beta2': 0.95, 'adam_epsilon': 1e-08, 'max_grad_norm': 1.0, 'num_train_epochs': 1.0, 'max_steps': -1, 'lr_scheduler_type': 'cosine', 'lr_scheduler_kwargs': {}, 'warmup_ratio': 0.0, 'warmup_steps': 200, 'log_level': 'info', 'log_level_replica': 'info', 'log_on_each_node': True, 'logging_dir': '/fsx_0/checkpoints/mm10/MM10-Stage2-70B/MH21_70B_336px_exp30d_docvqa_unfreeze_lr1_n128/evals_py/eval_ai2d_checkpoint-18800',

In [196]:
import torch

In [216]:
checkpoint_path = "/fsx_0/checkpoints/mm10/MM10-Stage2-70B/MH21_70B_336px_exp32a_lr1_n128/checkpoint-8000"
with open(f"{checkpoint_path}/training_args.bin", "rb") as fd:
    training_args = torch.load(fd, weights_only=False)
    
model_args = training_args.model_args.to_dict()
data_args = training_args.data_args
training_args = training_args.to_dict()
data_args.instruction_model_type

'TikTokv5ChatFormat'

In [219]:
model_args['lm_mh_tokenizer_version']

'tiktoken_v5'